# Advanced OOP
Here, I follow the LinkedIn Learning course [Advanced Python: Object-Oriented
Programming](https://www.linkedin.com/learning/advanced-python-object-oriented-programming/advanced-object-oriented-programming-oop?resume=false&u=72605090)
by Miki Tebeka and try the code. Here is the [course repo](https://github.com/LinkedInLearning/advanced-python-object-oriented-programming-4510177) (it is very good). I have added quite some additional explanations from ChatGPT, that I needed to understand, and also some additional exercises.

## Part 1: Attribute Access

### Class attribute access

In [ ]:
# what can go wrong with class attributes

class Player:
    num_players = 0  # Class attribute

    def __init__(self, name):
        self.name = name
        self.num_players += 1 # intention: updating class attribute
        print('self:', self.num_players)


p1 = Player('Parzival')
print('Player:', Player.num_players) 
# maybe surprise(not so much actually) : class attribute was not updated; that is
# because we had updated the instance attribute, not the class attribute

self: 1
Player: 0


In [ ]:
# internally, `__getattr__` does that:
def find_attribute(obj, attr):
    if attr in obj.__dict__:
        print(f'found {attr} in instance')
        return obj.__dict__[attr]

    if attr in obj.__class__.__dict__:
        print(f'found {attr} in class')
        return obj.__class__.__dict__[attr]

    for cls in obj.__class__.__mro__:
        if attr not in cls.__dict__:
            continue
        print(f'found {attr} in parent {cls.__name__!r}')
        return cls.__dict__[attr]

    # TODO: here we could manipulate __getattr__, use descriptors ... 

    raise AttributeError(attr)

class VM:
    version = '1.2.3'


class A1(VM):
    cpu_family = 'arm64'

    def __init__(self, id):
        self.id = id
        self.state = 'running'


a1 = A1('9e99929')

print(find_attribute(a1, 'id'))
print(find_attribute(a1, 'cpu_family'))
print(find_attribute(a1, 'version'))
print(find_attribute(a1, 'nic')) # expected AttributeError

found id in instance
9e99929
found cpu_family in class
arm64
found version in parent 'VM'
1.2.3


AttributeError: nic

### Using Properties

In [ ]:
class LineItem:
    def __init__(self, sku: str, price: float, amount: int):
        self.sku = sku # calls sku function (the one decorated with @sku.setter)
        self.price = price
        self.amount = amount

    @property  # 1st use case: computed property
    def value(self):
        return self.price * self.amount

    @property  # 2nd use case: getter
    def sku(self):
        return self._sku
    
    @sku.setter  # 2nd use case: setter
    def sku(self, value):
        value = value.strip()
        if not value:
            raise ValueError(f'empty sku: {value!r}')
        self._sku = value


li = LineItem('esp32', 1.34, 10)
print(li.value)

13.4


In [8]:
li.sku = ' ' # raises ValueError while setting

### Attribute access manipulation

- `object.__getattribute__` --> defines the regular attribute access (dot-notation)
    - to be handled with caution: if we overwrite it poorly, we could cause infinite
      recursion
- `object.__getattr__` --> is the fallback behaviour; its called only if the regular
  attribute access fails to find something (`object.__getattribute__` raises an
  `AttributeError`)
    -  useful for implementing dynamic or computed attributes without interfering with
       normal behavior

In [37]:
# example from course, but it's quite a riddle, if you're not experienced with databases

class Proxy:
    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, attr):
        value = getattr(self._obj, attr) # getattr is the build-in __getattr__ from the original obj
        print(f'{attr} -> {value!r}') # value is 0, for whatever reason ...
        return value

import sqlite3

conn = sqlite3.connect(':memory:')
proxy = Proxy(conn)
n = proxy.total_changes 
print(n)

proxy.close()
# I don't get this

total_changes -> 0
0
close -> <built-in method close of sqlite3.Connection object at 0x76069126fa60>


In [ ]:
# let's try to come up with my own example (experimental)

from sklearn.pipeline import Pipeline

class AdvancedPipeline(Pipeline):
    def __init__(self, new_param):
        self.new_param = new_param
        self.new_attribute = 0

    def __getattr__(self, attr):
        if attr:
            raise AttributeError(f"Alert: {attr} needs to evaluate to False.")
        return attr
    
    def __setattr__(self, attr, value):
        super().__setattr__(attr, value) #  needs to be called on super instead of self.attr to avoid RecursionError
        return self


instance = AdvancedPipeline(new_param = "something")

print(instance.new_attribute) # should not raise, because __getattr__ is only called if the attribute is NOT found

instance.new_attribute = 1
print(instance.new_attribute) # should not raise, because __getattr__ is only called if the attribute is NOT found

instance.another_attribute = 0
print(instance.another_attribute) # should not raise, because we have just set the attribute

instance.another_attribute = 1
print(instance.another_attribute) # should not raise, because we have just set the attribute

print(instance.yet_another_attribute) # should raise, because we have not created nor set the attribute



0
1
0
1


AttributeError: Alert: yet_another_attribute needs to evaluate to False.

- learning, again:
    - `__getattr__` is only invoked if the attribute is not found **at all** in the
      usual places: instance dictionary, class attributes, or parent classes.
    - if the attribute exists (set previously or defined on class), Python returns its
    value directly and never calls `__getattr__`

- it is totally useless in determining a value an attribute has

- but it can serve last-resort hook to provide a value or behavior when an attribute is
  missing

In [35]:
# let's apply this in a new example

class MyClass():
    def __getattr__(self, attr):
        if attr == "that_common_attribute_":
            return "Yep, that is what you were looking for!"
        else:
            raise AttributeError(f"`{attr}` does not exist. Did you mean `{self}.that_common_attribute_`?")
        
instance = MyClass()
instance.that_common_attribute_

'Yep, that is what you were looking for!'

In [ ]:
instance.that_other_attribute_some_people_ask_for_
# I would have to write a nice `__str__` and a `__repr__` for the plotting of `self`

AttributeError: `that_other_attribute_some_people_ask_for_` does not exist. Did you mean `<__main__.MyClass object at 0x7606912579b0>.that_common_attribute_`?

### Reducing memory with `__slots__`

- removes the `__dict__` attribute from (instances/classes)
- in scikit-learn excluded to work with any child from BaseEstimator

### Name mangling

- attributes with two leading underscores (e.g.,` __attr`) are automatically renamed to
  include the class name to avoid conflicts with the attribute names of the inheriting
  classes

In [41]:
class Base:
    def __init__(self):
        self.__hidden = 42

class Sub(Base):
    def __init__(self):
        super().__init__()
        self.__hidden = 99  # creates a different attribute _Sub__hidden


print(Sub.__dict__) # prints class attributes
print("")
print(Sub().__dict__) # prints instance attributes

{'__module__': '__main__', '__init__': <function Sub.__init__ at 0x760690eb1940>, '__doc__': None}

{'_Base__hidden': 42, '_Sub__hidden': 99}


class attributes
- `__module__`: name of the module in which the class was defined, or instance `__main__`
- `__init__`, or any other defined function
- `__doc__`: docstring of the class
- possible class attributes (not applicable here)

instance attributes
- attributes defined in `__init__` are always instance specific
- here we see '_Base__hidden'and '_Sub__hidden' because of the name mangling

### Descriptors

- live on class level
- need `__get__` and `__set__` defined
- mostly used for validation (see below for what this means)

In [45]:
class Validator:
    attr_name = '_validators'

    def __set_name__(self, cls, name): # `name` is variable name of instance, `cls` is class name (Price)
        print(f'__set_name__: {name=}')
        validator_name = self.__class__.__name__
        self._key = f'{validator_name}_{name}' # creating unique keys per instance

    def __get__(self, inst, cls):
        print(f'__get__: {inst=}, {cls=}')
        if inst is None:  # access via class (??)
            return self

        values = getattr(inst, Validator.attr_name, {})
        return values.get(self._key)

    def __set__(self, inst, value):
        print(f'__set__: {inst=}, {value=}')
        self.validate(value)
        values = getattr(inst, Validator.attr_name, None)
        if values is None:
            values = {}
            setattr(inst, Validator.attr_name, values)
        values[self._key] = value

    def validate(self, value):
        """This is to be overridden by subclasses."""
        raise NotImplementedError()


class Price(Validator):
    def validate(self, value):
        if value <= 0:
            raise ValueError(f'negative price: {value!r}')

class Trade:
    open = Price()
    close = Price()

    def __init__(self, open, close):
        self.open = open
        self.close = close
        

t1 = Trade(133.2, 147.5)
print(t1.__dict__)

__set_name__: name='open'
__set_name__: name='close'
__set__: inst=<__main__.Trade object at 0x760691255310>, value=133.2
__set__: inst=<__main__.Trade object at 0x760691255310>, value=147.5
{'_validators': {'Price_open': 133.2, 'Price_close': 147.5}}


In [46]:
print(Trade.open)  # Class level.
print(t1.open)  # Instance level.

t1.close = 147.2

t1.close = -2

__get__: inst=None, cls=<class '__main__.Trade'>
__get__: inst=<__main__.Trade object at 0x760691255310>, cls=<class '__main__.Trade'>
133.2
__set__: inst=<__main__.Trade object at 0x760691255310>, value=147.2
__set__: inst=<__main__.Trade object at 0x760691255310>, value=-2


ValueError: negative price: -2

--> that course example is unaccessible!

- working with ChatGPT to understand this better:

    - Descriptors are tools to customize attribute access

    - validation here means: before accepting and storing a value, check that it meets
      some condition

    - another way I now understood it is: validation is a term used in backend
      development; it means setting constraints on how a value can be set in a database

    - if someone sets `product.price = value`, we want to make sure it is positive

    - `Positive` class is a gatekeeper for setting values

In [34]:
# more minimal example from ChatGPT:
class Positive:
    def __set_name__(self, owner, name):
        self.name = name  # store attribute name (like 'price')

    def __get__(self, instance, owner):
        return instance.__dict__[self.name] # `getattr(instance, self.name)` would work as well

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError(f"{self.name} must be > 0")
        instance.__dict__[self.name] = value


class Product:
    price = Positive()  # descriptor used here

    def __init__(self, price):
        self.price = price


p = Product(10)
print(p.price)
p.price = -5 # raises ValueError: price must be > 0

10


ValueError: price must be > 0

- descriptors are set up when the class is created (not instantiated)

step-by-step

1. Class body of `Product` is executed, which creates a single instance of `Positive()`, and assigns it to the class attribute `price`
    - `Product.__dict__['price']` is now the `Positive()` instance

2. because `Positive` is a descriptor, Python automatically calls its
        `__set_name__` method
        - it stores the string `"price"` inside the descriptor so that it knows which
        attribute name it’s managing; This is optional but helpful

3. we instantiate Product: `p = Product(10)`
    - this calls `self.price = 10` in its `__init__`

4. but this doesn't call `self.__setattr__` directly; Instead, since `Product.price` is
    a descriptor, the descriptor’s `__set__` method is triggered like this:
    `Positive.__set__(self=Product.price, instance=p, value=10)`

5. the descriptor's `__set__` does the validation (check) and sets the value via the
   product instance's dict: `instance.__dict__[self.name] = value`, so like that:
   `p.__dict__['price'] = 10`

6. we run `print(p.price)`, which triggers the descriptor's `__get__` method like this:
   `Positive.__get__(self=Product.price, instance=p, owner=Product)`; which returns the
   value for price stored at the Product instance `p` (which is 10)

7. when we try to set the price badly (`p.price = -5`) the descriptor's `__set__` method
   raises on validation

--> THAT step-by_step really made it for me!


- descriptor that only have a `__get__` method (not a `__set__` method) are called non-data descriptors

    - commonly used for computed properties, like methods or cached values

In [ ]:
# minimal example for a non-data descriptor
class InchesToCm:
    def __get__(self, instance, owner):
        return instance._inches * 2.54

class Ruler:
    cm = InchesToCm()

    def __init__(self, inches):
        self._inches = inches

r = Ruler(10)
print(r.cm)

25.4


Exercise: Create a `NonEmptyString` Descriptor

Create a descriptor class `NonEmptyString` that ensures an attribute always contains a non-empty string. If someone tries to set it to an empty string or non-string value, raise a `ValueError`.

Then, use this descriptor in a class `User` with one attribute: `name`

Example behaviour: 

```
u = User("Alice")
print(u.name)     # should print "Alice"

u.name = ""       # should raise ValueError: name must be a non-empty string
u.name = None     # should raise ValueError
```

In [15]:
class NonEmptyString():
    """Descriptor"""

    def __set_name__(self, owner, name):
        self.name = name
    
    def __set__(self, instance, value):
        if value == "" or not isinstance(value, str):
            raise ValueError(f"`value` should be string and should not be empty, got `value={value}`.")
        instance.__dict__[self.name] = value # it's the descriptors' `self.name`

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name)


class User():
    name = NonEmptyString()

    def __init__(self, name):
        self.name = name # this setting calls `NonEmptyString.__set__()`

u = User("Alice")
print(u.name)

u.name=''

Alice


ValueError: `value` should be string and should not be empty, got `value=`.

Exercise: Create a descriptor called `PositiveInteger` that ensures an attribute is
always a positive integer. If someone tries to assign zero, a negative number, or a
non-integer (like a float or string), raise a `ValueError`.

Use it in a class `Person` that has an `age` attribute.

In [22]:
class PositiveInteger():
    """Descriptor"""

    def __set_name__(self, owner, name):
        self.name = name

    def __set__(self, instance, value):
        if not isinstance(value, int) or value < 0: # I allow infants ;)
            raise ValueError("`age` must be a positive integer.")
        instance.__dict__[self.name] = value

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name)


class Person():
    age = PositiveInteger() # setting the descriptor as a class attribute

    def __init__(self, age):
        self.age = age

person = Person(age=40)
person.age # 40

person = Person(age=-1)

ValueError: `age` must be a positive integer.

Course challenge: Code a class `Colors` that dynamically accesses color values from a
`color_dict`.

- I didn't understand what that meant (if I define the dict as a class attribute, isn't
  it dynamic, too?)

- ChatGPT explained:
    - What makes it dynamic is that the class or instance doesn't have those attributes
      defined explicitly.
    - When you access an attribute that doesn't exist on the instance or class, Python
      calls `__getattr__` and then  you look up the attribute name in a dictionary
      (which can be changed at runtime), and return the corresponding value on the fly.

- I am not 100 % sure I understand the difference between dynamic and normal, but I will
  give this a try:

In [32]:
color_dict = {
    'red': 0xFF0000,
    'green': 0x00FF00,
    'blue': 0x0000FF,
}

class Colors:
    """Dynamically get color from color_dict"""

    def __getattr__(self, attribute):
        if attribute in color_dict.keys():
            return color_dict.get(attribute)
        raise AttributeError(f"{self.__class__.__name__}.{attribute} doesn't exist.")

colors = Colors()

print(f'green: {colors.green:06X}')
print(f'red: {colors.red:06X}')
print(f'blue: {colors.blue:06X}')
colors.yellow

green: 00FF00
red: FF0000
blue: 0000FF


AttributeError: Colors.yellow doesn't exist.

- goal of dynamic attribute access is that we don't store too much data on the class(m)emory efficiency), flexibility, cleaner namespaces (no polluting `__dict__` with tons of static attributes)